In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash_table
import plotly.figure_factory as ff

### Data preparation

In [2]:
dfr= pd.read_csv('Data/Flight Dataset.csv')

In [3]:
dfr['ARR_DT']=dfr['ARR_DT'].map(lambda x: dt.datetime.strptime(x, '%Y%m%d:%H%M%S'))
dfr['DEP_DT']=dfr['DEP_DT'].map(lambda x: dt.datetime.strptime(x, '%Y%m%d:%H%M%S'))
dfr['flty']= dfr['DEP_DT'].dt.year
dfr['flt_ym']= dfr['DEP_DT'].dt.strftime('%Y-%m')

#### For OVV

In [4]:
nwyear_paxrevflt= dfr[['NETWORK', 'flty', 'revisedPAX', 'PREV', 'FLT_NO']].groupby(
    ['NETWORK', 'flty']).agg(
    flt_count= ('FLT_NO','count'), 
    prev_total= ('PREV','sum'), 
    pax_total= ('revisedPAX','sum'))
nwyear_paxrevflt.reset_index(inplace= True)


In [5]:
nwyear_paxrevflt.to_csv('Data/nwyear_paxrevflt.csv', index= False)

In [7]:
# country-year-pax-rev-flt
rdfrenet= pd.read_csv('Data/CountryReNet.csv')
ctryear_paxrevflt= dfr[['country_route', 'flty', 'revisedPAX', 'PREV', 'FLT_NO']].groupby(
    ['country_route', 'flty']).agg(
    flt_count= ('FLT_NO','count'), 
    prev_total= ('PREV','sum'), 
    pax_total= ('revisedPAX','sum'))
ctryear_paxrevflt.reset_index(inplace= True)

ctryear_paxrevflt =pd.merge(ctryear_paxrevflt, rdfrenet, on='country_route', how= 'left')

In [8]:
ctryear_paxrevflt.to_csv('Data/ctryear_paxrevflt.csv', index= False)

In [9]:
regyear_paxrevflt= ctryear_paxrevflt[['Region', 'flty', 'pax_total', 'prev_total', 'flt_count']].groupby(
    ['Region', 'flty']).agg(
    flt_count= ('flt_count','sum'), 
    prev_total= ('prev_total','sum'), 
    pax_total= ('pax_total','sum'))
regyear_paxrevflt.reset_index(inplace= True)
    

In [10]:
regyear_paxrevflt.to_csv('Data/regyear_paxrevflt.csv', index= False)

In [11]:
sankeyall= dfr[['country_route', 'flty', 'revisedPAX', 'PREV', 'FLT_NO']].groupby(
    ['country_route', 'flty']).agg(
    flt_count= ('FLT_NO','count'), 
    prev_total= ('PREV','sum'), 
    pax_total= ('revisedPAX','sum'))
sankeyall.reset_index(inplace= True)
sankeyall= pd.merge(sankeyall, rdfrenet, on='country_route', how= 'left')

In [12]:
sankeyall.to_csv('Data/sankeyall.csv', index= False)

In [13]:
indcol= ['flt_count', 'prev_total', 'pax_total', 'exbag_rev_total', 'cargow_total', 
          'cargo_rev_total', 'mail_rev_total', 'avg_loadrate', 'avgPaxRev']
inddict={'flt_count':'Flights', 'prev_total':'Revenue', 'prev_yq_total':'YQ revenue', 
         'exbag_rev_total': 'Exbag revenue', 'cargow_total':'Cargo Weight', 
         'cargo_rev_total': 'Cargo revenue', 'mail_rev_total': 'Mail revenue', 'pax_total': 'Passengers', 
         'avg_loadrate': 'Average load rate', 'avgPaxRev': 'Average revenue per Pax'}
indaggdict={'flt_count':'sum', 'prev_total':'sum', 'prev_yq_total':'sum', 
         'exbag_rev_total': 'sum', 'cargow_total':'sum', 
         'cargo_rev_total': 'sum', 'mail_rev_total': 'sum', 'pax_total': 'sum', 
         'avg_loadrate': 'avg', 'avgPaxRev': 'avg'}

#### For COUNTRY LEVEL ANALYSIS

In [14]:

ctrym_allind= dfr[['country_route', 'flt_ym','FLT_NO','PREV','EX_BAG_REV',
       'CARGO','CARGO_REV','MAIL_REV','revisedPAX',
          'rvload_rate']].groupby(['country_route', 'flt_ym']).agg(
    pax_total= ('revisedPAX','sum'),
    prev_total= ('PREV','sum'),
    
    flt_count= ('FLT_NO','count'), 
     
    exbag_rev_total= ('EX_BAG_REV','sum'),
    cargow_total= ('CARGO', 'sum'), 
    cargo_rev_total= ('CARGO_REV','sum'), 
    mail_rev_total= ('MAIL_REV','sum'), 
     
    avg_loadrate= ('rvload_rate', 'mean'))
ctrym_allind['avgPaxRev']= ctrym_allind['prev_total']/ ctrym_allind['pax_total']
ctrym_allind.reset_index(inplace= True)

In [15]:
ctrym_allind.to_csv('Data/ctrym_allind.csv', index= False)

In [16]:
dfintrank= pd.DataFrame()
dfintrank['Rank']= range(1,15)

for i, val in enumerate(indcol):
    indagg= indaggdict[val]
    col_i= inddict[val]
    dfpri= ctrym_allind[['country_route', val]]
    dfpri.columns= ['keyindex', 'aggcol']
    if indagg== 'sum':
        dftest= dfpri.groupby(by= 'keyindex').agg({'aggcol':'sum'}).sort_values(by='aggcol', ascending= False).reset_index()
    else:
        dftest= dfpri.groupby(by= 'keyindex').agg({'aggcol':'mean'}).sort_values(by='aggcol', ascending= False).reset_index()
    dfintrank[col_i]= list(dftest['keyindex'])
    
dfintrank.to_csv('Data/intRankingtable.csv', index= False)

In [17]:
dfrintdep= dfr.loc[dfr.ORI !='SGN',:]
intoriayear_paxrevflt= dfrintdep[['ORI', 'flty', 'revisedPAX', 'PREV', 'FLT_NO']].groupby(
    ['ORI', 'flty']).agg(
    flt_count= ('FLT_NO','count'), 
    prev_total= ('PREV','sum'), 
    pax_total= ('revisedPAX','sum'))
intoriayear_paxrevflt.reset_index(inplace= True)
intoriayear_paxrevflt.to_csv('Data/intoriayear_paxrevflt.csv', index= False)

#### For ROUTE ANALYSIS

In [19]:
dfrdom= dfr

In [20]:
sortedpaxroute= dfrdom[['city_roundroute','revisedPAX']].groupby('city_roundroute').agg(pax= ('revisedPAX', 'sum')).sort_values(by= 'pax', ascending= False)
sortedpaxroute.reset_index(inplace= True)


In [21]:
sortedpaxroute.to_csv('Data/sortedpaxroute.csv', index= False)

In [22]:
domrouteyear_paxrevflt= dfrdom[['city_roundroute', 'flty', 'revisedPAX', 'PREV', 'FLT_NO']].groupby(
    ['city_roundroute', 'flty']).agg(
    flt_count= ('FLT_NO','count'), 
    prev_total= ('PREV','sum'), 
    pax_total= ('revisedPAX','sum'))
domrouteyear_paxrevflt.reset_index(inplace= True)

In [23]:
domrouteyear_paxrevflt.to_csv('Data/domrouteyear_paxrevflt.csv', index= False)

In [24]:
domrouteym_allind= dfr[['city_roundroute', 'flt_ym','FLT_NO','PREV','EX_BAG_REV',
       'CARGO','CARGO_REV','MAIL_REV','revisedPAX',
          'rvload_rate']].groupby(['city_roundroute', 'flt_ym']).agg(
    flt_count= ('FLT_NO','count'), 
    prev_total= ('PREV','sum'), 
    exbag_rev_total= ('EX_BAG_REV','sum'),
    cargow_total= ('CARGO', 'sum'), 
    cargo_rev_total= ('CARGO_REV','sum'), 
    mail_rev_total= ('MAIL_REV','sum'), 
    pax_total= ('revisedPAX','sum'), 
    avg_loadrate= ('rvload_rate', 'mean'))
domrouteym_allind['avgPaxRev']= domrouteym_allind['prev_total']/ domrouteym_allind['pax_total']
domrouteym_allind.reset_index(inplace= True)

In [25]:
domrouteym_allind.to_csv('Data/domrouteym_allind.csv', index= False)

In [26]:
oriayear_paxrevflt= dfrdom[['ORI', 'flty', 'revisedPAX', 'PREV', 'FLT_NO']].groupby(
    ['ORI', 'flty']).agg(
    flt_count= ('FLT_NO','count'), 
    prev_total= ('PREV','sum'), 
    pax_total= ('revisedPAX','sum'))
oriayear_paxrevflt.reset_index(inplace= True)

In [27]:
oriayear_paxrevflt.to_csv('Data/oriayear_paxrevflt.csv', index= False)

In [28]:
activedom=pd.DataFrame(columns=['domroute','first_flt', 'last_flt'])
first_flt=[]
last_flt=[]
for i in sortedpaxroute['city_roundroute']:
    first_flt.append(dfrdom[dfrdom['city_roundroute']==i].DEP_DT.min())
    last_flt.append(dfrdom[dfrdom['city_roundroute']==i].DEP_DT.max())
activedom['domroute']= sortedpaxroute['city_roundroute']
activedom['first_flt']= first_flt
activedom['last_flt']= last_flt


In [29]:
activedom.to_csv('Data/activedom.csv', index= False)

##### State colors

In [30]:
colorseq= px.colors.qualitative.Antique+px.colors.qualitative.Vivid+px.colors.qualitative.Safe+px.colors.qualitative.Bold+px.colors.qualitative.Pastel+px.colors.qualitative.Prism+px.colors.qualitative.Set3
regioncolors= dict(zip(ctryear_paxrevflt.Region.unique(),px.colors.qualitative.Antique[0:4]))
ctrcolors= dict(zip(ctryear_paxrevflt.country_route.unique(), colorseq[0:14]))
domrcolors= dict(zip(sortedpaxroute['city_roundroute'], colorseq[0:48]))

### I. OVERVIEW

#### Fig1: Overview with Sankey 

In [31]:
# Select indicator: pax, rev, flt
def ovv_sankey_fig1(ovv_slidery, ovv_sankeyind):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#006666', '#009999', '#0099CC','#006699', '#CC9900', '#CC9966', '#CC9999', 
                    '#CCCCCC', '#CCCC99', '#CCCC66', '#FFCC66', '#FF9900', '#FF6666', '#CC3333', 
                    '#999966', '#993366', '#669999', '#666666', '#339999', '#336666', '#CC66CC', 
                    '#FFCC00', '#FFCC99', '#996600', '#4B8BBE','#306998','#FFE873','#FFD43B','#646464',
                   '#999933', '#99CCCC', '#FFCC00']
    df= sankeyall[sankeyall.flty== ovv_slidery]
    cat_cols= ['Network', 'Region', 'country_route']
    value_cols= ovv_sankeyind
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    #labelList = list(dict.fromkeys(labelList))
    
    # define colors 
    colorList = []
    for idx, colorNum in enumerate(labelList):
        # define colors based on number of levels
        #colorList = colorList + [colorPalette[idx]]*colorNum
        colorList = colorList + [colorPalette[idx]]
    
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).sort_values(by=['source','target','count'],ascending= False).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    node= node = dict(
          pad = 15,
          thickness = 80,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        )
    link= dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
    fig1= go.Figure(go.Sankey(link= link, node= node))
    fig1.update_layout(
        title= 'Overview country route Network of VNA by %s' %(ovv_sankeyind),
        font_size= 12
    )
    fig1.update_layout(
        hovermode="x",
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0")

    return fig1

In [32]:
ovv_sankey_fig1(2016, 'pax_total')

#### Fig2: Overview with bubble (year) time slider 

In [ ]:
#Choose color:
fig = px.colors.qualitative.swatches()
fig.show()
#print(px.colors.qualitative.Antique)

In [33]:
def ovv_bubble_fig2(ovv_bubbleyear):
    dfig2= ctryear_paxrevflt[(ctryear_paxrevflt.flty== ovv_bubbleyear) #& (ctryear_paxrevflt.country_route != 'Vietnam')
                            ]
    
    fig2= px.scatter(dfig2, x= 'pax_total', y= 'prev_total',
                     size='flt_count', color= 'Region', hover_name= 'country_route', #text= 'country_route',
                    labels={'pax_total': inddict['pax_total'], 'Region': "<b>Region</b>",
                           'prev_total': inddict['prev_total']},
                     color_discrete_map={k: regioncolors[k] for k in dfig2.Region.unique()},
                     #color_discrete_sequence= px.colors.qualitative.Antique, 
                     size_max= 55)
    #fig2.update_traces(textposition= 'top left')
    #fig2.update_layout(transition_duration= 500)
    
    fig2.update_traces(hovertemplate="<b>%{hovertext}</b><br>"+ "Passengers: %{x:.3s}<br>"+ 
                       "Revenue: %{y:.3s}<br>"+ "Flights: %{marker.size:.0f}<extra></extra>")
    fig2.update_layout(
        title= 'VNA performance by country in %s' %(str(ovv_bubbleyear)),
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
    fig2.update_layout(
        legend_title_font_size=14)
    fig2.update_xaxes(fixedrange=True)
    fig2.update_yaxes(fixedrange=True)
    return fig2

#print(ovv_bubble_fig2(2016).data[0].hovertemplate)

#### Fig3: Overview with pie chart 

In [34]:

def ovv_pie_fig3(ovv_pieyear):
    dfig3= regyear_paxrevflt[regyear_paxrevflt.flty== ovv_pieyear]
    fig3= make_subplots(rows= 1, cols= 2,
                        specs= [[{'type':'domain'}, {'type': 'domain'}]]
                       )
    # Make color :D
    #Opt1: simply drop marker_colors= px.colors.qualitative.Antique inside .add_traces if no need to assign specific color to each criteria
    #Opt2: create dict and maker arg instead of marker_colors:
    crit=dfig3.Region.unique()
    colors = np.array([''] * len(crit), dtype = object)
    for i in np.unique(crit):
        colors[np.where(crit == i)] = regioncolors[str(i)]
    
    fig3.add_traces(go.Pie(labels= dfig3.Region, values= dfig3.pax_total, textinfo= 'percent', 
                           insidetextorientation='radial', 
                           name= 'total passengers <br>in %s' %(str(ovv_pieyear)),
                           #sort= False,
                           marker= {'colors':colors}),
                    1,1)
    fig3.add_trace(go.Pie(labels= dfig3.Region, values= dfig3.prev_total, textinfo= 'percent',
                          insidetextorientation='radial',
                          name= 'total revenue <br>in %s' %(str(ovv_pieyear)),
                          #sort= False,
                          marker= {'colors':colors}),
                    1,2)
    # Use 'hole' to create donut-like
    fig3.update_traces(hole=.4, hoverinfo= 'label+value',
                      hovertemplate= "<b>%{label}</b><br>"+ "%{value:.3s}"
                      )
    #fig3.update_layout(transition_duration= 500)
    fig3.update_layout(
        title_text= 'VNA marketshare by Region in %s' %(str(ovv_pieyear)),
        # Add annotation in the center
        annotations= [dict(text='Passengers', x= 0.14, xanchor= 'left', y=0.5, font_size=15, showarrow= False),
                 dict(text='Revenue', x=0.83, xanchor= 'right', y=0.5, font_size=15, showarrow=False)],
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        showlegend= True,
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1.34),
    )
    
    return fig3
 
    
    #'<b>%{country_route}</b> %{x}<br><br>'+ 'Average load rate: %{y:.1%}<extra></extra>'

#### Fig4: Overview for each Region in timeline (year) - input selector by hover data of Fig3 

In [35]:
def ovv_stackedbar_fig4(ovv_barreg):
    dfig4= regyear_paxrevflt[(regyear_paxrevflt.Region== ovv_barreg) & 
                             (regyear_paxrevflt.flty != 2020)]
    fig4= make_subplots(rows= 3, cols= 1, shared_xaxes= True, vertical_spacing=0.07)
    
    fig4.add_traces(go.Scatter(name= 'Revenue',
                           x= dfig4.flty,
                           y= dfig4.prev_total,
                               mode= 'lines+markers',
                           marker=dict(color= regioncolors[ovv_barreg]),
                           opacity= 1),
                    1,1)
    fig4.add_traces(go.Scatter(name= 'Flights',
                           x= dfig4.flty,
                           y= dfig4.flt_count,
                               mode= 'lines+markers',
                           marker=dict(color= regioncolors[ovv_barreg]),
                           opacity= 0.6),
                    2,1)
    fig4.add_traces(go.Scatter(name= 'Passengers',
                           x= dfig4.flty,
                           y= dfig4.pax_total,
                               mode= 'lines+markers',
                           marker=dict(color= regioncolors[ovv_barreg]),
                           opacity= 0.3),
                    3,1)
    fig4.update_xaxes(tickformat="d")
    fig4.update_traces(hovertemplate= "%{x}"+": %{y:.3s}")
    fig4.update_layout(
        title_text= 'Revenue, flights & passengers of %s ' %(str(ovv_barreg)),
        # Add annotation in the center
        #hovermode= 'x',
        height= 720,
        width=1000,
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        showlegend= True,
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1),
    )
    
    return fig4
    

In [36]:
ovv_stackedbar_fig4('North East Asia')

### II. COUNTRY

#### Tab1: Ranking int country by ind 

In [ ]:
dash_table.DataTable(
    data=intranktab.to_dict('record'),
    columns=[{'id': c, 'name': c} for c in intranktab],
    fixed_rows={'headers': True},
    style_as_list_view=True,
    editable=False,
    style_table={
        'height': 300,
        'overflowX': 'auto'
    },
    style_header={"backgroundColor": "#f8f5f0", "fontWeight": "bold"},
    style_cell={'height': 'auto', 'textAlign': 'center', 'padding': '8px',
                'minWidth': '90px', 'width': '90px', 'maxWidth': '90px',
                'whiteSpace': 'normal'},
)


#### Fig5: Int monthly timeline comparision (select multi-crt, single ind) 

In [37]:
def int_monthline_fig5(ctrlistfig5, indfig5):
    dfig5= ctrym_allind[ctrym_allind.country_route.isin(ctrlistfig5)]
    #labely_fig5= inddict[indfig5]
    
    fig5 = px.line(dfig5, x='flt_ym', y=indfig5,
                   color='country_route',
                   labels={indfig5: inddict[indfig5], 'country_route': "<b>Country</b>",
                           'flt_ym': 'Monthly Timeline'},
                   color_discrete_map={k: ctrcolors[k] for k in ctrlistfig5},
                   title='{} by {} \
                  <br>in Jan2015 - Feb2020</b>'.format(", ".join(ctrlistfig5),inddict[indfig5]))

    fig5.update_xaxes(rangeslider_visible=True,
                      dtick="M1",
                      tickformat="%b\n%Y", #tickfont=dict(size=10)
                     )
    fig5.update_traces(hovertemplate="<b>%{x}</b><br>"+ "%s: " %(inddict[indfig5])+"%{y:.3s}<br>")
    #fig5.update_layout(legend_title_font_size=14)
    fig5.update_layout(
        hovermode='x',
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
    #fig1.update_traces(hovertemplate=None)
    # fig1.update_layout(margin=dict(l=5, r=10, t=25, b=20)) <extra></extra>"
    return fig5


In [38]:
int_monthline_fig5(['Japan'], 'pax_total')

#### Fig6: Int TSA seasonal yearline country (select single crt-ind)

In [39]:
def int_yearline_fig6(ctrfig6, indfig6):
    dfig6= ctrym_allind.loc[ctrym_allind.country_route== ctrfig6, :]
    dfig6.flt_ym= dfig6.flt_ym.map(lambda x: dt.datetime.strptime(x,'%Y-%m'))
    dfig6['flty'] = dfig6.flt_ym.dt.year
    dfig6['fltm'] = dfig6['flt_ym'].dt.strftime('%b')
    
    avgmindlist=dfig6[dfig6.flty !=2020][['fltm',indfig6]].groupby('fltm').agg(
    avg=(indfig6,'mean')).sort_values(by='avg', ascending= False).reset_index()

    
    
    Statement = f'**Peak season:** \ ' \
                f'* The highest average {inddict[indfig6]} month is {avgmindlist.fltm[0]} \'' \
                f'* The second highest average {inddict[indfig6]} month is {avgmindlist.fltm[1]}\'' \
                f'**Low season:**\''\
                f'* The lowest average {inddict[indfig6]} month is {avgmindlist.fltm[11]}'
    fig6= px.line(dfig6, x='fltm', y=indfig6,
                   color='flty', #markers= True,
                   labels={indfig6: inddict[indfig6], 'flty': "<b>Year</b>",
                           'fltm': 'Monthly Timeline'},
                   color_discrete_sequence=px.colors.qualitative.Bold,
                   title='{} {} Seasonal analysis\
                  <br>in Jan2015 - Feb2020</b>'.format(ctrfig6, inddict[indfig6]))

    fig6.update_xaxes(#rangeslider_visible=True,
        dtick="M1",
        #tickformat="%b\n", #ticklabelmode='period'
        #tickfont=dict(size=10)
        )
    fig6.update_traces(
        hovertemplate="<b>%{x}</b><br>"+ "%s: " %(inddict[indfig6])+"%{y:.3s}<br>",
        marker_symbol= 'circle',
        

    )
    #fig6.update_layout(legend_title_font_size=14)
    fig6.update_layout(
        hovermode='x',
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
    #fig1.update_traces(hovertemplate=None)
    # fig1.update_layout(margin=dict(l=5, r=10, t=25, b=20)) <extra></extra>"
    print(Statement)
    return fig6


In [40]:
def int_yearline_fig6(indfig6, slidery, ctrfig6="Japan"):
    dfig6 = ctrym_allind.loc[ctrym_allind.country_route == ctrfig6, :]
    dfig6.flt_ym = dfig6.flt_ym.map(lambda x: dt.datetime.strptime(x, '%Y-%m'))
    dfig6['flty'] = dfig6.flt_ym.dt.year
    dfig6['fltm'] = dfig6['flt_ym'].dt.strftime('%b')
    avgmindlist = dfig6[dfig6.flty != 2020][['fltm', indfig6]].groupby('fltm').agg(
        avg=(indfig6, 'mean')).sort_values(by='avg', ascending=False).reset_index()


    Highestm= f'The highest month of {inddict[indfig6]} of {ctrfig6} route is {avgmindlist.fltm[0]} '
    Shighestm= f'The second highest month of {inddict[indfig6]} of {ctrfig6} route is {avgmindlist.fltm[1]}'
    Lowestm= f'The lowest month {inddict[indfig6]} of {ctrfig6} route is {avgmindlist.fltm[11]}'

    fig6 = go.Figure()

    for i, y in enumerate(dfig6.flty.unique()):
        fig6.add_traces(go.Scatter(name=str(y),
                                    x=dfig6[dfig6.flty == y]['fltm'],
                                    y=dfig6[dfig6.flty == y][indfig6],
                                    mode='lines+markers',
                                    marker=dict(color=px.colors.qualitative.Bold[i]),
                                    opacity=1))

    fig6.update_xaxes(  # rangeslider_visible=True,
        dtick="M1",
        # tickformat="%b\n", #ticklabelmode='period'
        # tickfont=dict(size=10)
    )
    fig6.update_traces(
        hovertemplate="<b>%{x}</b><br>" + "%s: " % (inddict[indfig6]) + "%{y:.3s}<br>",
        marker_symbol='circle'

    )
    # fig6.update_layout(legend_title_font_size=14)
    fig6.update_layout(
        title='{} Seasonal analysis of {} country route\
                     <br>in Jan2015 - Dec2019</b>'.format(inddict[indfig6], ctrfig6),
        #hovermode='x',
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
    return fig6

In [41]:
int_yearline_fig6('pax_total', 2015, 'Japan')

#### Fig12: Top countries - bar chart 

In [42]:
def int_bartopctr_fig12(yearfig12, indfig12):
    dintfig12= ctryear_paxrevflt[ctryear_paxrevflt.country_route != 'Vietnam']
    dfig12= dintfig12.loc[dintfig12.flty== yearfig12, ['country_route',indfig12]].sort_values(by= indfig12, ascending= False)
    
    fig12= go.Figure(
        data=[go.Bar(
            name= 'International countries',
            x= dfig12.country_route,
            y= dfig12[indfig12],
            marker_color= "#AE8F6F",
            opacity= 0.8)])
    fig12.update_traces(
        hovertemplate="<b>%{x}</b><br>"+ "%s in %s: " %(inddict[indfig12], str(yearfig12))+"%{y:.3s}<br>",    
    )
    fig12.update_layout(
        title="Top down international countries by %s in %s" %(inddict[indfig12], str(yearfig12)),
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1),
    )
    fig12.update_traces(textangle= -60)
    return fig12

In [43]:
int_bartopctr_fig12(2016, 'pax_total')

#### Fig13: Top 20 int departure airport (ori) bar chart

In [44]:
def int_bartopdepa_fig13(yearfig13, indfig13):
    dfig13= intoriayear_paxrevflt.loc[intoriayear_paxrevflt.flty== yearfig13, ['ORI',indfig13]].sort_values(by= indfig13, ascending= False)
    dfig13= dfig13.iloc[:20,:]
    fig13= go.Figure(
        data=[go.Bar(
            name= 'International Departure Airport',
            x= dfig13.ORI,
            y= dfig13[indfig13],
            marker_color= "#004687",
            opacity= 0.8)])
    fig13.update_traces(
        hovertemplate="<b>Depart from %{x}</b><br>"+ "%s in %s: " %(inddict[indfig13], str(yearfig13))+"%{y:.3s}<br><extra></extra>",    
    )
    fig13.update_layout(
        title="Top 20 international departure airport by %s in %s" %(inddict[indfig13], str(yearfig13)),
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1),
    )
    fig13.update_traces(textangle= -60)
    return fig13
        

In [45]:
int_bartopdepa_fig13(2016, 'flt_count')

### III. ROUTE

#### Fig7: Dom timeline operation route gantt chart 

In [46]:

activedom.rename(columns={'domroute':'Task', 'first_flt':'Start','last_flt':'Finish'}, inplace= True)
fig7= ff.create_gantt(activedom,  colors= "#AE8F6F", width= 900, height= 900)
fig7.update_xaxes(rangeslider_visible=True)
fig7.update_layout(
        title='Domestic route operation history Jan2015-Feb2020',
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
fig7.show()

#### Fig8: Top 30 route bar chart (select year-ind)

In [ ]:
def dom_bartoproute_fig8(yearfig8, indfig8):
    dfig8= domrouteyear_paxrevflt.loc[domrouteyear_paxrevflt.flty== yearfig8, ['city_roundroute',indfig8]].sort_values(by= indfig8, ascending= False)
    dfig8= dfig8.iloc[:30,:]
    fig8= go.Figure(
        data=[go.Bar(
            name= 'Domestic route',
            x= dfig8.city_roundroute,
            y= dfig8[indfig8],
            marker_color= "#AE8F6F",
            opacity= 0.8)])
    fig8.update_traces(
        hovertemplate="<b>%{x}</b><br>"+ "%s in %s: " %(inddict[indfig8], str(yearfig8))+"%{y:.3s}<br>",    
    )
    fig8.update_layout(
        title="Top 30 domestic round-route by %s in %s" %(inddict[indfig8], str(yearfig8)),
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1),
    )
    fig8.update_traces(textangle= -60)
    return fig8
        

#### Fig9: Dom roundroute monthly timeline comparision (select multi-domroute, single ind)

In [ ]:
def dom_monthline_fig9(domrlistfig9, indfig9):
    dfig9= domrouteym_allind[domrouteym_allind.city_roundroute.isin(domrlistfig9)]
    #labely_fig5= inddict[indfig5]
    
    fig9 = px.line(dfig9, x='flt_ym', y=indfig9,
                   color='city_roundroute',
                   labels={indfig9: inddict[indfig9], 'city_roundroute': "<b>Domestic round route</b>",
                           'flt_ym': 'Monthly Timeline'},
                   color_discrete_map={k: domrcolors[k] for k in domrlistfig9},
                   title='{} by {} \
                  <br>in Jan2015 - Feb2020</b>'.format(", ".join(domrlistfig9),inddict[indfig9]))

    fig9.update_xaxes(rangeslider_visible=True,
                      dtick="M1",
                      tickformat="%b\n%Y", #tickfont=dict(size=10)
                     )
    fig9.update_traces(hovertemplate="<b>%{x}</b><br>"+ "%s: " %(inddict[indfig9])+"%{y:.3s}<br>")
    #fig5.update_layout(legend_title_font_size=14)
    fig9.update_layout(
        hovermode='x',
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
    #fig1.update_traces(hovertemplate=None)
    # fig1.update_layout(margin=dict(l=5, r=10, t=25, b=20)) <extra></extra>"
    return fig9

#### Fig10:  Top 10 dom departure airport (ori) bar chart (select year-ind)

In [ ]:
def dom_bartopdepa_fig10(yearfig10, indfig10):
    dfig10= oriayear_paxrevflt.loc[oriayear_paxrevflt.flty== yearfig10, ['ORI',indfig10]].sort_values(by= indfig10, ascending= False)
    dfig10= dfig10.iloc[:10,:]
    fig10= go.Figure(
        data=[go.Bar(
            name= 'Departure Airport',
            x= dfig10.ORI,
            y= dfig10[indfig10],
            marker_color= "#004687",
            opacity= 0.8)])
    fig10.update_traces(
        hovertemplate="<b>Depart from %{x}</b><br>"+ "%s in %s: " %(inddict[indfig10], str(yearfig10))+"%{y:.3s}<br><extra></extra>",    
    )
    fig10.update_layout(
        title="Top 10 domestic departure airport by %s in %s" %(inddict[indfig10], str(yearfig10)),
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1),
    )
    fig10.update_traces(textangle= -60)
    return fig10
        

#### Fig11:  Dom TSA seasonal yearline roundroute (select single roundroute-ind)

In [39]:
dfig11= domrouteym_allind.loc[domrouteym_allind.city_roundroute== 'Danang : Tokyo', :]
dfig11.flt_ym= dfig11.flt_ym.map(lambda x: dt.datetime.strptime(x,'%Y-%m'))
dfig11['flty'] = dfig11.flt_ym.dt.year
dfig11['fltm'] = dfig11['flt_ym'].dt.strftime('%b')
avgmindlist=dfig11[['fltm','pax_total']].groupby('fltm').agg(
avg=('pax_total','mean')).sort_values(by='avg', ascending= False).reset_index()


In [49]:
def int_yeardomrline_fig11statement(indfig11, domrfig11):
    dfig11 = domrouteym_allind.loc[domrouteym_allind.city_roundroute==domrfig11, :]
    dfig11.flt_ym = dfig11.flt_ym.map(lambda x: dt.datetime.strptime(x, '%Y-%m'))
    dfig11['flty'] = dfig11.flt_ym.dt.year
    dfig11['fltm'] = dfig11['flt_ym'].dt.strftime('%b')
    avgmindlist = dfig11[dfig11.flty != 2020][['fltm', indfig11]].groupby('fltm').agg(
        avg=(indfig11, 'mean')).sort_values(by='avg', ascending=False).reset_index()
    orderm= list(avgmindlist.fltm)


    Highestm= f'The highest month of {inddict[indfig11]} of route {domrfig11} is {orderm[0]}'
    Shighestm= f'The second highest month of {inddict[indfig11]} of route {domrfig11} is {orderm[1]}'
    Lowestm= f'The lowest month of {inddict[indfig11]} of route {domrfig11} is {orderm[-1]}'

    fig11 = go.Figure()

    for i, y in enumerate(dfig11.flty.unique()):
        fig11.add_traces(go.Scatter(name=str(y),
                                    x=dfig11[dfig11.flty == y]['fltm'],
                                    y=dfig11[dfig11.flty == y][indfig11],
                                    mode='lines+markers',
                                    marker=dict(color=px.colors.qualitative.Bold[i]),
                                    opacity=1))

    fig11.update_xaxes(  # rangeslider_visible=True,
        dtick="M1",
        # tickformat="%b\n", #ticklabelmode='period'
        # tickfont=dict(size=10)
    )
    fig11.update_traces(
        hovertemplate="<b>%{x}</b><br>" + "%s: " % (inddict[indfig11]) + "%{y:.3s}<br>",
        marker_symbol='circle'

    )
    # fig6.update_layout(legend_title_font_size=14)
    fig11.update_layout(
        title='{} Seasonal analysis of {} round route\
                  <br>in Jan2015 - Dec2019</b>'.format(inddict[indfig11], domrfig11),
        #hovermode='x',
        font={"color": "darkslategray"},
        paper_bgcolor="white",
        plot_bgcolor="#f8f5f0",
        legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
    # fig1.update_traces(hovertemplate=None)
    # fig1.update_layout(margin=dict(l=5, r=10, t=25, b=20)) <extra></extra>"
    return fig11